This notebook contains the Execution of the statistical models used in the thesis (pipeline function). The model functions are defined. The datasets for the analysis are created given data has been preprocessed (downsampled, filtered). The data is originally structured in tseries each containing a set of trials that belong together. The tseries are combined to leverage all data of the desired type (complex_sub, pure). Arrays that store the CV-metrics are created and saved depending on the CV method. For each setting (delay, duration of spectrogram (T), number of components (R)) the models are fitted and CV-scores are computed insample and out-of-sample.
The data is stored at FIAS. These files should be executed once the auditorycoding directory folder (as on the fias server) is placed in the same directory as these files. A "temp" folder should be created to store temporary CV-folds is mode=drive is used. Also a "Linear Mappings" directory should be created.

In [1]:
import numpy as np
from tqdm.notebook import tqdm
from sklearn.cross_decomposition import CCA as CCAsklearn
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import make_pipeline
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_percentage_error
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pickle
from pls_regression import PLSRegression as PLSRegressionEdited
from pls_regression import CCA as CCAEdited
from pls_regression import PLSCanonical as PLSCanonicalEdited
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import os

from preprocessing_final import preprocessing, reverse_preprocessing_img, generate_masks, cv_split_LOO, select_stimulus_subset, preprocessing_pure, cv_split_trial, cv_split_newstim, preprocessing_mps
from audio_processing_final import create_spectrograms, plot_spectrogram

In [2]:
def standardize(X):
    X_c = (X - np.mean(X, axis=0) )/ np.std(X, axis=0)
    return X_c

class CCA():
    """Implementation of Canonical correlation analysis following the Paper:
    https://dl.acm.org/doi/10.1145/3136624, https://doi.org/10.1145/3136624
    n_components sets the number of components in the model, mode the solution to the CCA problem
    eig should be used, ridge sets the value deducted from the diagonal of correlation matrices.
    """
    def __init__(self, n_components=2, mode="eig", ridge=False):
        self.mode = mode
        self.n_components = n_components
        self.ridge = ridge
        
    def standardize(self, X):
        X_c = (X - np.mean(X, axis=0) )/ np.std(X, axis=0)
        return X_c
    
    def fit(self, X, Y):
        N = X.shape[0]
        Cxx = (1 / N) * X.T @ X
        Cyy = (1 / N) * Y.T @ Y
        Cxy = (1 / N) * X.T @ Y
        Cyx = (1 / N) * Y.T @ X
        
        if self.mode == "eig":
            if self.ridge == False:
                R = np.linalg.inv(Cyy) @ Cyx @ np.linalg.inv(Cxx) @ Cxy
                eigvals, eigvecs = np.linalg.eig(R)
                self.correlations = np.sqrt(eigvals)
                self.wy = eigvecs[:,:self.n_components]
                self.wx = (np.linalg.inv(Cxx) @ Cxy @ self.wy) / self.correlations[:self.n_components]

            if self.ridge != False:
                print("ridge")
                R = np.linalg.inv(Cyy - self.ridge * np.identity(Cyy.shape[0])) @ Cyx @ np.linalg.inv(Cxx - self.ridge * np.identity(Cxx.shape[0])) @ Cxy
                eigvals, eigvecs = np.linalg.eig(R)
                eigvals, eigvecs = np.real(eigvals), np.real(eigvecs)
                self.correlations = np.sqrt(eigvals)
                self.wy = eigvecs[:,:self.n_components]
                self.wx = (np.linalg.inv(Cxx- self.ridge * np.identity(Cxx.shape[0])) @ Cxy @ self.wy) / self.correlations[:self.n_components]
            
            return self.wx, self.wy, self.correlations
            
        elif self.mode == "gen_eig":
            print("not implemented")
            pass
            
        elif self.mode == "svd":
            CCxx = np.linalg.cholesky(Cxx)
            CCyy = np.linalg.cholesky(Cyy)

            U, s, V = np.linalg.svd(np.linalg.inv(CCxx) @ Cxy @ np.linalg.inv(CCyy), full_matrices=False)
            """S = np.zeros((4,3))
            for i in range(len(s)):
                S[i,i] = s[i]"""
            self.wx = np.linalg.inv(CCxx) @ U[:,:self.n_components]
            self.wy = np.linalg.inv(CCyy) @ V[:,:self.n_components].T
            self.correclations = s
            
            return self.wx, self.wy, self.correclations
        
    def predict(self, X):
        Y_hat = X @ self.wx @ self.wy.T
        return Y_hat
        
class RRR():
    """Reduced rank regression model adapted from https://github.com/berenslab/patch-seq-rrr
    ridge sets regularization parameter (alpha).
    """
    def __init__(self, ridge=False):
        self.ridge = ridge


    def fit(self, X, Y):
        U,s,V = np.linalg.svd(X, full_matrices=False)
        if self.ridge != False:
            self.B = V.T @ np.linalg.inv(np.diag(s**2) + self.ridge * np.identity(s.shape[0])) @ np.diag(s) @ U.T @ Y

        elif self.ridge == False:
            self.B = V.T @ np.diag(s/(s**2)) @ U.T @ Y

        self.U,self.s,self.V = np.linalg.svd(X@self.B, full_matrices=False)


    def predict(self, X, rank):
        w = self.B @ self.V.T[:,:rank]
        v = self.V.T[:,:rank]

        pos = np.argmax(np.abs(v), axis=0)
        flips = np.sign(v[pos, range(v.shape[1])])
        v = v * flips
        w = w * flips
        return X @ w @ v.T


In [3]:
def pipeline(method, dataset, subset=None, subset_predict=None,
             relevance_cutoff=0.25,
             ds=4, 
             input_durations=[1.5], 
             delays=[0], 
             components=range(1,51,2), 
             cv_mode="trial", 
             cv_splits=5,
             save=True,
             reverse_direction=False,
             single_input=False,
             ridge=False,
             datamode="filtered",
             mps=False):
    """This function contains the main computations for the regression models.
    method = [PCR, LR, RRR, CCA, PLS]
    dataset = [pure, complex_sub]
    subset and subset_predict should remain as None
    relevance_cutoff = see create_masks in preprocessing
    ds = downsampling factor
    input_durations = list of duration of spectrograms to be tested (sec)
    delays = list of delays b/w stimulus and reaction
    components = list of number of components (R) to keep in model
    cv_mode = [default (random CV), trial (trial CV), newstim (stimulus CV)]
    cv_splits = 5 but has no effect on CV methods
    save has no effect
    reverse_direction yields backwards predictions if True
    single input represents pure tones as single feature instead of spectrogram
    ridge = alpha hyperparameter for LR (MLR) and RRR (RRRR)
    datamode should be set to filtered but deconv works
    mps changes spectrograms to modulation power spectrograms
    
    """
    # Select tseries that are part of dataset
    spectrograms = create_spectrograms(FFT_overlap=0, plot=False, scaling="decibels")
    tseries = []
    stimuli = []
    numTrials = 0
    if "pure" in dataset:
        tseries.extend([f"tseries_{i}" for i in [28, 33, 34, 39]])
        stimulus = np.load("auditorycoding/F189/tseries_28/stimulus.npy")
        stim_set = set([s[0] + 203 for s in stimulus])
        if single_input == False:
            stim_set.difference_update({213, 214, 215})
        stimuli.extend(stim_set)
        numTrials += 32
    if "complex_full" in dataset:
        tseries.extend([f"tseries_{i}" for i in [21, 24, 27, 31, 36, 38]])
        stimulus = np.load("auditorycoding/F189/tseries_21/stimulus.npy")
        stimuli.extend(set([s[0] for s in stimulus]))
    if  "complex_sub" in dataset:
        tseries.extend([f"tseries_{i}" for i in [23, 26, 29, 32, 37]])
        stimulus = np.load("auditorycoding/F189/tseries_23/stimulus.npy")
        stimuli.extend(set([s[0] for s in stimulus]))
        numTrials += (15 - 7)
    if "mov" in dataset:
        tseries.extend([f"tseries_{i}" for i in [25, 30]])
        # ???? TBD
        stimulus = np.load("auditorycoding/F189/tseries_28/stimulus.npy")
        stimuli.extend([s[0] + 203 for s in stimulus])


    if subset_predict != None:
        stimuli = subset_predict
    
    # Create the matrices to store evaluation metrics, the size of shich depends on direction,
    # dataset, datamode, input representation and CV method
    # Metrics available are explained variance (evs), mean absolute percentage error (mape)
    #, and mean squared error (mse)
    if reverse_direction == False:
        if datamode == "deconv":
            output_size = {0.25: 3053} #Size of Y-imageds depending on mask
        elif datamode != "deconv":
            output_size = {0: 3347, 0.25: 3072, 0.5:2648, 0.75: 2240}

    elif reverse_direction == True:
        if single_input == True:
            output_size = {relevance_cutoff: 1}

        elif single_input == False:
            if mps == False:
                output_size = {relevance_cutoff: int(15 * max(input_durations)) * len(spectrograms[0][1])}
            elif mps == True:
                output_size = {relevance_cutoff: int(77*101)}
    
    if method not in ["CCA", "PLS", "RRR", "PCR", "EN", "PLSCAN"]:
        components = [0] # Linear regression has no components

    if cv_mode == "default":
        evs_scores = np.zeros((len(delays), len(input_durations), len(components), cv_splits, output_size[relevance_cutoff]))
        mape_scores = np.zeros((len(delays), len(input_durations), len(components), cv_splits, output_size[relevance_cutoff]))
        mse_scores = np.zeros((len(delays), len(input_durations), len(components), cv_splits, output_size[relevance_cutoff]))

        evs_scores_insample = np.zeros((len(delays), len(input_durations), len(components), cv_splits, output_size[relevance_cutoff]))
        mape_scores_insample = np.zeros((len(delays), len(input_durations), len(components), cv_splits, output_size[relevance_cutoff]))
        mse_scores_insample = np.zeros((len(delays), len(input_durations), len(components), cv_splits, output_size[relevance_cutoff]))

    elif cv_mode == "loo":
        evs_scores = np.zeros((len(delays), len(input_durations), len(components), len(stimuli), output_size[relevance_cutoff]))
        mape_scores = np.zeros((len(delays), len(input_durations), len(components), len(stimuli), output_size[relevance_cutoff]))   
        mse_scores = np.zeros((len(delays), len(input_durations), len(components), len(stimuli), output_size[relevance_cutoff]))

        evs_scores_insample = np.zeros((len(delays), len(input_durations), len(components), len(stimuli), output_size[relevance_cutoff]))
        mape_scores_insample = np.zeros((len(delays), len(input_durations), len(components), len(stimuli), output_size[relevance_cutoff]))   
        mse_scores_insample = np.zeros((len(delays), len(input_durations), len(components), len(stimuli), output_size[relevance_cutoff]))    

    elif cv_mode == "trial":
        evs_scores = np.zeros((len(delays), len(input_durations), len(components), numTrials, output_size[relevance_cutoff]))
        mape_scores = np.zeros((len(delays), len(input_durations), len(components), numTrials, output_size[relevance_cutoff]))
        mse_scores = np.zeros((len(delays), len(input_durations), len(components), numTrials, output_size[relevance_cutoff]))

        evs_scores_insample = np.zeros((len(delays), len(input_durations), len(components), numTrials, output_size[relevance_cutoff]))
        mape_scores_insample = np.zeros((len(delays), len(input_durations), len(components), numTrials, output_size[relevance_cutoff]))
        mse_scores_insample = np.zeros((len(delays), len(input_durations), len(components), numTrials, output_size[relevance_cutoff]))

    elif cv_mode == "newstim":
        evs_scores = np.zeros((len(delays), len(input_durations), len(components), 11, output_size[relevance_cutoff]))
        mape_scores = np.zeros((len(delays), len(input_durations), len(components), 11, output_size[relevance_cutoff]))   
        mse_scores = np.zeros((len(delays), len(input_durations), len(components), 11, output_size[relevance_cutoff]))

        evs_scores_insample = np.zeros((len(delays), len(input_durations), len(components), 11, output_size[relevance_cutoff]))
        mape_scores_insample = np.zeros((len(delays), len(input_durations), len(components), 11, output_size[relevance_cutoff]))   
        mse_scores_insample = np.zeros((len(delays), len(input_durations), len(components), 11, output_size[relevance_cutoff]))       

    # for each setting we create datamatrices by stacking the input and output matrices from each tseries ontop of each other    
    max_components = max(components)
    for i, d in tqdm(enumerate(delays)):
        for j, duration in tqdm(enumerate(input_durations)):
            input_seconds = duration
            delay = d
            if single_input == True:
                X_cut, Y_cut, ft_cut, frame_idx_cut, frame_trial_idx_cut = preprocessing_pure(tseries[0],
                relevance_cutoff=relevance_cutoff, ds=ds, delay=delay, mode=datamode)[2:]
                
                for series in tseries[1:]:
                    Xi_cut, Yi_cut, fti_cut, framei_idx_cut, framei_trial_idx_cut = preprocessing_pure(series,
                    relevance_cutoff=relevance_cutoff, ds=ds, delay=delay, mode=datamode)[2:]
                    X_cut = np.vstack((X_cut, Xi_cut))
                    Y_cut = np.vstack((Y_cut, Yi_cut))
                    frame_idx_cut = np.append(frame_idx_cut, framei_idx_cut)
                    frame_trial_idx_cut = np.append(frame_trial_idx_cut, framei_trial_idx_cut + frame_trial_idx_cut.max())

            elif single_input == False:
                if mps == False:
                    X_cut, Y_cut, ft_cut, frame_idx_cut, frame_trial_idx_cut = preprocessing(tseries[0], spectrograms=spectrograms,
                    relevance_cutoff=relevance_cutoff, input_seconds=input_seconds, ds=ds, delay=delay, mode=datamode)[2:]

                    for series in tseries[1:]:
                        Xi_cut, Yi_cut, fti_cut, framei_idx_cut, framei_trial_idx_cut = preprocessing(series, spectrograms=spectrograms,
                        relevance_cutoff=relevance_cutoff, input_seconds=input_seconds, ds=ds, delay=delay, mode=datamode)[2:]
                        X_cut = np.vstack((X_cut, Xi_cut))
                        Y_cut = np.vstack((Y_cut, Yi_cut))
                        frame_idx_cut = np.append(frame_idx_cut, framei_idx_cut)
                        frame_trial_idx_cut = np.append(frame_trial_idx_cut, framei_trial_idx_cut + frame_trial_idx_cut.max())
                elif mps == True:
                    X_cut, Y_cut, ft_cut, frame_idx_cut, frame_trial_idx_cut = preprocessing_mps(tseries[0],
                    relevance_cutoff=relevance_cutoff, ds=ds, delay=delay, mode=datamode)[2:]

                    for series in tseries[1:]:
                        Xi_cut, Yi_cut, fti_cut, framei_idx_cut, framei_trial_idx_cut = preprocessing_mps(series,
                        relevance_cutoff=relevance_cutoff, ds=ds, delay=delay, mode=datamode)[2:]
                        X_cut = np.vstack((X_cut, Xi_cut))
                        Y_cut = np.vstack((Y_cut, Yi_cut))
                        frame_idx_cut = np.append(frame_idx_cut, framei_idx_cut)
                        frame_trial_idx_cut = np.append(frame_trial_idx_cut, framei_trial_idx_cut + frame_trial_idx_cut.max())
                
                #print(X_cut.shape, Y_cut.shape)
            if subset:
                X_cut, Y_cut, frame_idx_cut, frame_trial_idx_cut = select_stimulus_subset(X_cut, 
                                                                                          Y_cut, 
                                                                                          frame_idx_cut, 
                                                                                          frame_trial_idx_cut, 
                                                                                          subset)
            
            # Remove faulty trials

            if "complex_sub" in dataset:
                invalid_trials = [1,2,7,9,11,13,15]
                valid_trials = set(frame_trial_idx_cut).difference(invalid_trials)
                valid_idx = [a for a in range(len(frame_trial_idx_cut))
                             if any(frame_trial_idx_cut[a] == b for b in valid_trials)]
                X_cut = X_cut[valid_idx,:]
                Y_cut = Y_cut[valid_idx,:]
                frame_idx_cut = frame_idx_cut[valid_idx]
                frame_trial_idx_cut = frame_trial_idx_cut[valid_idx]
                
            
            if reverse_direction == True:
                X_cut, Y_cut = Y_cut, X_cut
            
            # Select CV method and fit model for each split
            if cv_mode == "default":
                splits_X, splits_Y = cv_split(X_cut, Y_cut)

            elif cv_mode == "loo":
                splits_X, splits_Y = cv_split_LOO(X_cut, Y_cut, frame_idx_cut, index_set=subset_predict, mode="drive")

            elif cv_mode == "trial":
                splits_X, splits_Y = cv_split_trial(X_cut, Y_cut, frame_trial_idx_cut, index_set=subset_predict, mode="drive")
                #splits_X, splits_Y = splits_X, splits_Y

            elif cv_mode == "newstim":
                splits_X, splits_Y = cv_split_newstim(X_cut, Y_cut, frame_idx_cut, frame_trial_idx_cut, index_set=subset_predict, mode="drive", reverse_direction=reverse_direction)
                
            
            for split in tqdm(range(len(splits_X))):
                if cv_mode == "default":
                    X_train, X_test = splits_X[split]
                    Y_train, Y_test = splits_Y[split]

                if cv_mode in ["loo", "trial", "newstim"]:
                    X_train = np.load(splits_X[split][0])
                    X_test = np.load(splits_X[split][1])

                    Y_train = np.load(splits_Y[split][0])
                    Y_test = np.load(splits_Y[split][1])


                # Z-Score Data
                X_scaler = StandardScaler().fit(X_train)
                Y_scaler = StandardScaler().fit(Y_train)

                
                
                # Potential tests for multicollinearity
                #eigvals = np.linalg.eigvals(X_scaler.transform(X_train).T @ X_scaler.transform(X_train))
                #print("Eigvals close to Zero: ", len(np.where(np.isclose(eigvals, 0))[0]))
                #sorted_eigvals =  np.sort(eigvals)
                #print("Sorted Eigvals: ",sorted_eigvals)

                # Train models
                if method == "PLS":
                    model = PLSRegressionEdited(n_components=min(max_components, X_train.shape[1]),scale=False)
                    model.fit(X_scaler.transform(X_train), Y_scaler.transform(Y_train))
                if method == "PLSCAN":
                    model = PLSCanonicalEdited(n_components=min(max_components, X_train.shape[1]),scale=False)
                    model.fit(X_scaler.transform(X_train), Y_scaler.transform(Y_train))
                if method == "RRR":
                    model = RRR(ridge=ridge)
                    model.fit(X_scaler.transform(X_train), Y_scaler.transform(Y_train))

                
                
                for l, c in enumerate(components):
                    # Predict test data for each number of components
                    # Out of sample metrics
                    
                    if method == "CCA":
                        model = CCA(n_components=c, mode="eig", ridge=ridge)
                        model.fit(X_scaler.transform(X_train), Y_scaler.transform(Y_train))
                        Y_pred = Y_scaler.transform(Y_test)
                    if method == "PCR":
                        model = model = make_pipeline(PCA(n_components=c), LinearRegression())
                        model.fit(X_scaler.transform(X_train), Y_scaler.transform(Y_train))
                        Y_pred = model.predict(X_scaler.transform(X_test))
                    elif method == "LR":
                        model = Ridge(alpha=ridge)
                        model.fit(X_scaler.transform(X_train), Y_scaler.transform(Y_train))
                        Y_pred = model.predict(X_scaler.transform(X_test))
                    elif method == "EN":
                        model = ElasticNet(alpha=c[0], l1_ratio=c[1], fit_intercept=False,selection="random")
                        model.fit(X_scaler.transform(X_train), Y_scaler.transform(Y_train))  
                        Y_pred = model.predict(X_scaler.transform(X_test))  
                    elif method in ["PLS", "RRR", "PLSCAN"]:    
                        Y_pred = model.predict(X_scaler.transform(X_test), rank=c)

                    evs = explained_variance_score(Y_test, Y_scaler.inverse_transform(Y_pred), multioutput="raw_values")
                    mape = mean_absolute_percentage_error(Y_test, Y_scaler.inverse_transform(Y_pred), multioutput="raw_values")
                    mse = mean_squared_error(Y_test, Y_scaler.inverse_transform(Y_pred), multioutput="raw_values")
                    
                    evs_scores[i,j,l,split,:] = evs
                    mape_scores[i,j,l,split,:] = mape
                    mse_scores[i,j,l,split,:] = mse
                    
                    if cv_mode == "loo":
                        # not used
                        np.save(f"LOO_Partitions/{method}_{'_'.join(dataset)}_subeset_{subset}_{delay}_{input_seconds}_{split}_{c}_{reverse_direction}_single{single_input}_true.npy", Y_test)
                        np.save(f"LOO_Partitions/{method}_{'_'.join(dataset)}_subeset_{subset}_{delay}_{input_seconds}_{split}_{c}_{reverse_direction}_single{single_input}_pred.npy", Y_scaler.inverse_transform(Y_pred))
                    
                    # In-sample metrics
                    if method == "PCR":
                        Y_pred = model.predict(X_scaler.transform(X_train))
                    if method == "CCA":
                        Y_pred = Y_scaler.transform(Y_train)
                    elif method == "LR":
                        Y_pred = model.predict(X_scaler.transform(X_train))
                    elif method == "EN":
                        Y_pred = model.predict(X_scaler.transform(X_train))
                        print(X_train.shape, Y_pred.shape, Y_train.shape) 
                    elif method in ["PLS", "RRR", "PLSCAN"]:    
                        Y_pred = model.predict(X_scaler.transform(X_train), rank=c)

                    evs = explained_variance_score(Y_train, Y_scaler.inverse_transform(Y_pred), multioutput="raw_values")
                    mape = mean_absolute_percentage_error(Y_train, Y_scaler.inverse_transform(Y_pred), multioutput="raw_values")
                    mse = mean_squared_error(Y_train, Y_scaler.inverse_transform(Y_pred), multioutput="raw_values")
                    
                    evs_scores_insample[i,j,l,split,:] = evs
                    mape_scores_insample[i,j,l,split,:] = mape
                    mse_scores_insample[i,j,l,split,:] = mse

            # Save scores of the model for each setting and split

            if reverse_direction == False:
                np.save(f"Linear Mapping/{method}_{'_'.join(dataset)}_subeset_{subset}_{cv_mode}_{reverse_direction}_single{single_input}_evs_ridge{ridge}.npy", evs_scores)
                np.save(f"Linear Mapping/{method}_{'_'.join(dataset)}_subeset_{subset}_{cv_mode}_{reverse_direction}_single{single_input}_mape_ridge{ridge}.npy", mape_scores)
                np.save(f"Linear Mapping/{method}_{'_'.join(dataset)}_subeset_{subset}_{cv_mode}_{reverse_direction}_single{single_input}_mse_ridge{ridge}.npy", mse_scores)

                np.save(f"Linear Mapping/{method}_{'_'.join(dataset)}_subeset_{subset}_{cv_mode}_{reverse_direction}_single{single_input}_evs_insample_ridge{ridge}.npy", evs_scores_insample)
                np.save(f"Linear Mapping/{method}_{'_'.join(dataset)}_subeset_{subset}_{cv_mode}_{reverse_direction}_single{single_input}_mape_insample_ridge{ridge}.npy", mape_scores_insample)
                np.save(f"Linear Mapping/{method}_{'_'.join(dataset)}_subeset_{subset}_{cv_mode}_{reverse_direction}_single{single_input}_mse_insample_ridge{ridge}.npy", mse_scores_insample)

            elif reverse_direction == True:
                np.save(f"Linear Mapping/{method}_{'_'.join(dataset)}_subeset_{subset}_{cv_mode}_{reverse_direction}_single{single_input}_duration{input_durations}_evs_ridge{ridge}.npy", evs_scores)
                np.save(f"Linear Mapping/{method}_{'_'.join(dataset)}_subeset_{subset}_{cv_mode}_{reverse_direction}_single{single_input}_duration{input_durations}_mape_ridge{ridge}.npy", mape_scores)
                np.save(f"Linear Mapping/{method}_{'_'.join(dataset)}_subeset_{subset}_{cv_mode}_{reverse_direction}_single{single_input}_duration{input_durations}_mse_ridge{ridge}.npy", mse_scores)

                np.save(f"Linear Mapping/{method}_{'_'.join(dataset)}_subeset_{subset}_{cv_mode}_{reverse_direction}_single{single_input}_duration{input_durations}_evs_insample_ridge{ridge}.npy", evs_scores_insample)
                np.save(f"Linear Mapping/{method}_{'_'.join(dataset)}_subeset_{subset}_{cv_mode}_{reverse_direction}_single{single_input}_duration{input_durations}_mape_insample_ridge{ridge}.npy", mape_scores_insample)
                np.save(f"Linear Mapping/{method}_{'_'.join(dataset)}_subeset_{subset}_{cv_mode}_{reverse_direction}_single{single_input}_duration{input_durations}_mse_insample_ridge{ridge}.npy", mse_scores_insample)                

            pickle.dump(model, open(f"Linear Mapping/{method}_{'_'.join(dataset)}_subeset_{subset}_{cv_mode}_delay{d}_duration{duration}_{reverse_direction}_single{single_input}_ridge{ridge}.p", "wb"))
            temp = os.listdir("temp/")
            for file_ in temp:
                os.remove(f"temp/{file_}")
                
    return evs_scores, mape_scores, mse_scores
